In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

In [2]:
# read the data
df = pd.read_csv("Resources/charity_data.csv")
df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# drop the 'EIN' and 'NAME' columns as they are non-beneficial ID columns
app_df = df.drop(["EIN", "NAME"], axis = 1)
app_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [4]:
# determine how many unique values in each columns
app_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [5]:
# look at "APPLICATION_TYPE" value counts for binning purposes
app_type = app_df["APPLICATION_TYPE"].value_counts()
app_type

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# choose a cutoff value and create a list of application types to be replaced
type_to_replace = list(app_type[app_type < 515].index)

# replace the app type above in the df
for app in type_to_replace:
    app_df["APPLICATION_TYPE"] = app_df["APPLICATION_TYPE"].replace(app, "Other")

# make sure binning was successful
app_df["APPLICATION_TYPE"].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [7]:
# look at "CLASSIFICATION" value counts for binning purposes
class_count = app_df["CLASSIFICATION"].value_counts()
class_count

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [8]:
# look at value counts > 1
class_list = list(class_count[class_count > 1])
class_list

[17326,
 6074,
 4837,
 1918,
 1883,
 777,
 287,
 194,
 116,
 114,
 104,
 95,
 75,
 58,
 50,
 36,
 34,
 32,
 32,
 30,
 20,
 18,
 16,
 15,
 15,
 14,
 11,
 10,
 10,
 9,
 9,
 7,
 6,
 6,
 6,
 5,
 5,
 3,
 3,
 3,
 2,
 2,
 2,
 2,
 2]

In [9]:
# choose a cutoff value and create a list of classifications to be replaced
class_to_replace = list(class_count[class_count < 1500].index)

# replace the classification above in the df
for classification in class_to_replace:
    app_df["CLASSIFICATION"] = app_df["CLASSIFICATION"].replace(classification, "Other")
    
# make sure binning was successful
app_df["CLASSIFICATION"].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [10]:
app_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [11]:
# convert categorical data to numeric with the get_dummies function
app_df = pd.get_dummies(app_df)
app_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [12]:
# split data into feautres and target arrays
x = app_df.drop('IS_SUCCESSFUL', axis = 1)
y = app_df['IS_SUCCESSFUL']

# split the data into training and testing
X_train, X_test, y_train, y_test = train_test_split(x, y, random_state = 41)

In [13]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [14]:
# compile, train, and evaluate the model
# define the model
nn = tf.keras.models.Sequential()

# 1st hidden layer
nn.add(tf.keras.layers.Dense(units = 80, activation = "relu", input_dim=43))

# 2nd hidden layer
nn.add(tf.keras.layers.Dense(units = 30, activation = "relu"))

# output layer
nn.add(tf.keras.layers.Dense(units = 1, activation = "sigmoid"))

# check the structure of the model
nn.summary() 

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 80)                3520      
                                                                 
 dense_1 (Dense)             (None, 30)                2430      
                                                                 
 dense_2 (Dense)             (None, 1)                 31        
                                                                 
Total params: 5,981
Trainable params: 5,981
Non-trainable params: 0
_________________________________________________________________


In [15]:
# compile the model
nn.compile(loss = "binary_crossentropy", optimizer = "adam", metrics = ["accuracy"])

In [16]:
# train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 1s 640us/step - loss: 0.5685 - accuracy: 0.7234
Epoch 2/100
804/804 [==============================] - 1s 643us/step - loss: 0.5537 - accuracy: 0.7319
Epoch 3/100
804/804 [==============================] - 1s 640us/step - loss: 0.5509 - accuracy: 0.7324
Epoch 4/100
804/804 [==============================] - 1s 651us/step - loss: 0.5489 - accuracy: 0.7331
Epoch 5/100
804/804 [==============================] - 1s 704us/step - loss: 0.5473 - accuracy: 0.7343
Epoch 6/100
804/804 [==============================] - 1s 677us/step - loss: 0.5469 - accuracy: 0.7340
Epoch 7/100
804/804 [==============================] - 1s 649us/step - loss: 0.5462 - accuracy: 0.7345
Epoch 8/100
804/804 [==============================] - 1s 682us/step - loss: 0.5456 - accuracy: 0.7345
Epoch 9/100
804/804 [==============================] - 1s 675us/step - loss: 0.5452 - accuracy: 0.7355
Epoch 10/100
804/804 [==============================] - 1s 695us/step - l

804/804 [==============================] - 1s 667us/step - loss: 0.5354 - accuracy: 0.7413
Epoch 80/100
804/804 [==============================] - 1s 656us/step - loss: 0.5353 - accuracy: 0.7411
Epoch 81/100
804/804 [==============================] - 1s 650us/step - loss: 0.5349 - accuracy: 0.7410
Epoch 82/100
804/804 [==============================] - 1s 651us/step - loss: 0.5351 - accuracy: 0.7416
Epoch 83/100
804/804 [==============================] - 1s 651us/step - loss: 0.5350 - accuracy: 0.7415
Epoch 84/100
804/804 [==============================] - 1s 660us/step - loss: 0.5345 - accuracy: 0.7412
Epoch 85/100
804/804 [==============================] - 1s 657us/step - loss: 0.5347 - accuracy: 0.7413
Epoch 86/100
804/804 [==============================] - 1s 668us/step - loss: 0.5346 - accuracy: 0.7414
Epoch 87/100
804/804 [==============================] - 1s 655us/step - loss: 0.5346 - accuracy: 0.7417
Epoch 88/100
804/804 [==============================] - 1s 670us/step - loss:

In [17]:
# evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose = 2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5601 - accuracy: 0.7238 - 205ms/epoch - 763us/step
Loss: 0.5601406693458557, Accuracy: 0.7238484025001526


In [18]:
# export model to HDF5 file
nn.save_weights("Model\charity_deep_learning.h5", overwrite=True, save_format=None, options=None)